In [39]:
#Exercise 1: write a function that determines the reciprocal of a number without using division
#Here we will find 1/e and show that it converges for x0 = 0.3 but not for x0 = 1.0

function recip(d, x)
    for i=1:100 x = x .* (2 - x .* d) end
    return x
end

e = exp(1)

println(recip(e, 0.3))
println(recip(e, 1.0))

0.36787944117144233

In [40]:
function sup_norm(x)                    #easier to read later code with this defined
    return maximum(abs(x))
end

function newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
    c1 = 1e-4                #Armijo condition constant
    c2 = 0.9                 #curvature condition constant
    
    armijo = new_f - f <= c1*alpha*(g'*p)[1,1]           #satisfies Armijo condition?
    curvature = abs(new_g'*p)[1,1] > -c2*(g'*p)[1,1]     #satisfies curvature condition?
    valley = (new_g'*p)[1,1] < 0                         
    return (armijo, curvature, valley)
end

function newtmin( obj, x0; maxIts=100, optTol=1e-6, BFGS=false)
# Minimize a function f using Newton’s method.
# obj: a function that evaluates the objective value, gradient, and Hessian at a point x, i.e.,
#    (f, g, H) = obj(x)
# x0: starting point.
# maxIts (optional): maximum number of iterations.
# optTol (optional): optimality tolerance based on ||grad(x)|| <= optTol*||grad(x0)||
# BFGS (optional): set as true to use BFGS algorithm ( = false by default)

    verbose = false
    
    its = 0;
    x = x0;
    
    if(BFGS)
        (f,g,_) = obj(x)
        n = size(g)[1]
        H_inv = eye(n)/norm(g)        #approximate Hessian as scaled identity
    else
        (f,g,H) = obj(x)     #evaluate gradient, et al
    end
      
    ngx0 = sup_norm(g)           #need to keep this value for stopping condition
    
    if(ngx0 > optTol^2)      #trap for low grad(x0)
        
        while(its < maxIts && sup_norm(g) > optTol*(ngx0))
            
            Hessian_modded = false
            
            if(BFGS)
                p = -H_inv * g
            else
                p = -H \ g           #compute descent direction
            end
            
            alpha = 1
            alpha_min = 0
            alpha_max = 1
            
            new_x = x + alpha*p  #trial step
            
            
            (new_f,new_g,_) = obj(new_x)
            
            (armijo, curvature, valley) = newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
                
            while(!armijo || !valley)
                
                if(!Hessian_modded && !BFGS)
                    #force the Hessian to be positive definite
                    
                    if(verbose) print("H"); end
                    
                    Hessian_modded = true
                
                    (V, S) = eig(H)                    #decompose H
                
                    if(minimum(V) < 0)                 #if not positive definite
                    
                        #V[V .<= 0] = maximum(V)        #do not explore areas of negative curvature
                        
                        V = V + abs(1.01*minimum(V))
                    
                        V_inv = diagm(1 ./ V)
                    
                        p = -S*V_inv*S'*g              #recalculate p    
                        
                    end
                
                else    #Hessian was already fixed
                
                    if(!armijo)
                        alpha_max = alpha                 #not enough decrease; try a smaller step
                        if(verbose) print("A"); end
                    elseif(!curvature)
                        if(verbose) print("C"); end
                        break                             #decrease is too slow to worry about hills/valleys
                    else
                        if(verbose) print("V"); end
                        alpha_min = alpha                 #going to land on a hill, try to speed over it
                    end
                                
                    alpha = 0.5*(alpha_max + alpha_min)   #try an intermediate value
                end
                
                new_x = x + alpha*p                   #trial step
            
                (new_f,new_g,_) = obj(new_x)      #evaluate new gradient, et al
                
                
                (armijo, curvature, valley) = newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
            
                if (alpha_max - alpha_min < 0.01)       #interval is too narrow
                    if(verbose) print("B"); end
                    break        #break out of loop; just take a step and hope for the best
                end
            
            end        #terminate linesearch
            
            
            #calculate Hessian
            if (BFGS)
                y = new_g - g
                s = alpha*p
                ro = 1 / (y'*s)[1,1]
                H_inv = (eye(n) - ro*s*y')*H_inv*(eye(n) - ro*y*s') + ro*s*s'    
            else
                (_,_,H) = obj(new_x)
            end
            
            x = new_x           #commit to the step
            f = new_f
            g = new_g
            
            Hessian_modded = false
            
            its = its + 1
            
        end
    
    else
        println("||grad(x0)|| is already < ", optTol^2)
    end
    
    return (x, its)
    
end

newtmin (generic function with 1 method)

In [43]:
using Toms566

#test the algorithm on Toms566

max_its = 2000
tol = 1e-6

println("Number of iterations for Newton/BFGS\n");

for number = 1:18
       
    p = Problem(number)

    (x, its) =
        newtmin(xk -> (p.obj(xk), p.grd(xk), p.hes(xk)), p.x0; maxIts = max_its, optTol = tol, BFGS = false)
                                    #newtmin(anonymous function, p.x0)
    (x_bfgs, its_bfgs) =
        newtmin(xk -> (p.obj(xk), p.grd(xk), p.hes(xk)), p.x0; maxIts = max_its, optTol = tol, BFGS = true)
    
    g = p.grd(x)
    g_bfgs = p.grd(x_bfgs)
    g0 = p.grd(p.x0)
    
    print("Problem #", number, ":  ")
    
    if(sup_norm(g)/sup_norm(g0) <= tol || sup_norm(g0) <= tol^2)
        print(its)
    else
        print("FAIL")
    end
    
    print(", ")
    
    if(sup_norm(g_bfgs)/sup_norm(g0) <= tol || sup_norm(g0) <= tol^2)
        print(its_bfgs)
    else
        print("FAIL")
    end
    
    println()
end

sleep(0.1)    #to keep output from bleeding into the next cell

Number of iterations for Newton/BFGS

Problem #1:  11, 28
Problem #2:  FAIL, 40
Problem #3:  2, 5
Problem #4:  88, 135
Problem #5:  13, 75
Problem #6:  17, 17
Problem #7:  FAIL, 56
Problem #8:  11, 18
Problem #9:  32, 113
Problem #10:  8, 28
Problem #11:  31, 36
Problem #12:  14, 51
Problem #13:  105, 62
Problem #14:  FAIL, 40
Problem #15:  FAIL, 49
Problem #16:  9, 14
Problem #17:  36, 35
Problem #18:  FAIL, 215
